# Making a playable chess game with numpy and python

In [ ]:
import numpy as np

In [2]:
#We shall represent the board as an 8x8 numpy array
Board = np.zeros((8, 8),  dtype=np.int) #np.int removes decimal point
Board

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

We shall represent the pieces by a 2 digit number:

The first number repsents the piece type.
Pawn = 1
Rook = 2
Knight = 3
Bishop = 4
Queen = 5
King = 6

The second represents if the piece is black or white. 
0 = White
1 = Black

e.g 
10 - is a white Pawn.
61 - is a black King

In [3]:
# Now we specify the starting position 


#Black's pieces 
Board[1][0:] = 11
Board[0][0] = 21
Board[0][1] = 31
Board[0][2] = 41
Board[0][3] = 51
Board[0][4] = 61
Board[0][5] = 41
Board[0][6] = 31
Board[0][7] = 21

#White's pieces 
Board[6][0:] = 10
Board[7][0] = 20
Board[7][1] = 30
Board[7][2] = 40
Board[7][3] = 50
Board[7][4] = 60
Board[7][5] = 40
Board[7][6] = 30
Board[7][7] = 20

In [4]:
Board

array([[21, 31, 41, 51, 61, 41, 31, 21],
       [11, 11, 11, 11, 11, 11, 11, 11],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10, 10, 10, 10, 10, 10, 10],
       [20, 30, 40, 50, 60, 40, 30, 20]])

# Here we stop nonsense moves


In [5]:
#Here we create a function to check input values are allowable
numbers = [str(i) for i in range(8)]
Allowable_squares = []
for i in numbers:
    for j in numbers:
        Allowable_squares.append(i+j)
        
def allowable_input(initial_square, target_square):
    if initial_square not in Allowable_squares:
        return False
    if target_square not in Allowable_squares:
        return False
    else:
        return True

In [6]:
#Checking a piece has been selected
def piece_selected(initial_square, target_square, chess_position):
    if chess_position[int(initial_square[0])][int(initial_square[1])] == 0:
        return False
    else:
        return True  

In [7]:
#checking player isnt attempting to take their own piece
def no_self_sacrifice(initial_square, target_square, chess_position):
    if chess_position[int(target_square[0])][int(target_square[1])] ==0:
        return True
    else:
        initial_square_colour = str(chess_position[int(initial_square[0])][int(initial_square[1])])[1]
        target_square_colour = str(chess_position[int(target_square[0])][int(target_square[1])])[1]
    
        if initial_square_colour == target_square_colour:
            return False
        else:
            return True

# We we start to define how pieces move

In [8]:
"""
I shall Group pieces together from easiest to hardest to write, depending on how they move.


GROUP 1: Depends only on immediate position, and other pieces do no influence how it moves.

- Knight, although the knight jumps around the board, it has no special rules, indeed it can move to any square
as long as not occupied by one of its own. 

- Bishop, Queen - These pieces also have no special rules however, they are contrained by their own pieces as they can't
jump unlie the knight

-  Rook - Although the rook can castle, we shall consider that a move the king initiates rather than the rook

GROUP 2: Depends on the move before:

- Pawn - Be wary of en passant! - Also have to consider promotions

GROUP 3: Depends on how pieces have moved throughout entire game.

- King,- These pieces can move in special ways via castling, both king and queenside, so we have to 
create functions that allow us to check if castling is legal, on either side: 1) King and Knight can't have already move. 
2)You can't castle into or through check. It seems best to write these pieces last as these moves depend and can be 
influnced by every other piece. Note, movement rules do not depend on position but rather if thay have already moved.

The King is the most complex pieces.
"""

"\nI shall Group pieces together from easiest to hardest to write, depending on how they move.\n\n\nGROUP 1: Depends only on immediate position, and other pieces do no influence how it moves.\n\n- Knight, although the knight jumps around the board, it has no special rules, indeed it can move to any square\nas long as not occupied by one of its own. \n\n- Bishop, Queen - These pieces also have no special rules however, they are contrained by their own pieces as they can't\njump unlie the knight\n\n-  Rook - Although the rook can castle, we shall consider that a move the king initiates rather than the rook\n\nGROUP 2: Depends on the move before:\n\n- Pawn - Be wary of en passant! - Also have to consider promotions\n\nGROUP 3: Depends on how pieces have moved throughout entire game.\n\n- King,- These pieces can move in special ways via castling, both king and queenside, so we have to \ncreate functions that allow us to check if castling is legal, on either side: 1) King and Knight can't h

# Knight

In [9]:
"""
In essence whatever the starting position of white, we either add or subtract 1 or 2 to each square value
If any are not on (0,7) then we remove them as they are not on the board
If they land on their own piece its not allowed
"""
def possible_knight_positions(initial_square, chess_position): #Note enter initial_square as a str
    
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
    #Possible knights moves
    x_addition = np.array([-2,-2,-1,-1,1,1,2,2])
    y_addition = np.array([1,-1,2,-2,2,-2,1,-1])
    possible_x = x_addition + xlabel
    possible_y = y_addition + ylabel
    
    tuple_of_possible_positions = list(zip(possible_x,possible_y))
    
    
    #Removing tuples not on the board
    acceptable_values = [0,1,2,3,4,5,6,7]
    result1 = [i for i in tuple_of_possible_positions if i[0] in acceptable_values]
    result2 = [i for i in result1 if i[1] in acceptable_values]
    
    
    #Removing tuples that represent taking own piece
    result3 = []
    for i in result2:
        string_of_i = str(i[0]) + str(i[1])
        if no_self_sacrifice(initial_square, string_of_i, chess_position) == True:
            result3.append(i)
    
   
    return result3

# Bishop

In [10]:
"""
The Bishop can move in 4 directions, as bishop traverses a diagonal the index increments either:
We call them:
UR: (-1,1)  #Upper Right
UL: (-1,-1) #Upper Left
LL: (1,-1)  #Lower Left
LR: (1,1)   #Lower Right
"""
def possible_bishop_positions(initial_square,chess_position):
    tuple_of_possible_positions = []
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
 
    
    #First we check the possibilities in UR (-1,1) 
    new_xlabel = xlabel
    new_ylabel = ylabel
    
    while True:
        new_xlabel -= 1
        new_ylabel += 1
        if new_xlabel < 0:
            break
        if new_ylabel <0:
            break
        if new_xlabel >7:
            break
        if new_ylabel >7:
            break   
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] != str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    #Then we check the possibilities in UL (-1,-1)
    new_xlabel = xlabel
    new_ylabel = ylabel
    while True:
        
        new_xlabel -= 1
        new_ylabel -= 1
        if new_xlabel < 0:
            break
        if new_ylabel <0:
            break
        if new_xlabel >7:
            break
        if new_ylabel >7:
            break   
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] != str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    
    
    
    #Then we check the possibilities in LL (1,-1)
    new_xlabel = xlabel
    new_ylabel = ylabel
    while True:
        
        new_xlabel += 1
        new_ylabel -= 1
        if new_xlabel < 0:
            break
        if new_ylabel <0:
            break
        if new_xlabel >7:
            break
        if new_ylabel >7:
            break  
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] != str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    #Then we check the possibilities in LR (1,1)
    new_xlabel = xlabel
    new_ylabel = ylabel
    while True:
        new_xlabel += 1
        new_ylabel += 1
        if new_xlabel <0:
            break
        if new_ylabel <0:
            break
        if new_xlabel >7:
            break
        if new_ylabel >7:
            break   
        
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] != str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    
    return tuple_of_possible_positions
    
    
    

In [11]:
possible_bishop_positions('62',Board)

[(5, 3), (4, 4), (3, 5), (2, 6), (1, 7), (5, 1), (4, 0)]

In [12]:
possible_bishop_positions('62',Board)

[(5, 3), (4, 4), (3, 5), (2, 6), (1, 7), (5, 1), (4, 0)]

# Rook


In [13]:
"""
The Rook can move in 4 directions:
We call them:
U: (-1,0)  #Upper Right
D: (1,0) #Upper Left
L: (0,-1)  #Lower Left
R: (0,1)   #Lower Right
"""
def possible_rook_positions(initial_square,chess_position):
    tuple_of_possible_positions = []
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
 
    
    #First we check the possibilities in U: (-1,0) #Upper Right
    new_xlabel = xlabel
    new_ylabel = ylabel
    
    while True:
        new_xlabel -= 1
        
        if new_xlabel < 0:
            break
       
        if new_xlabel >7:
            break
         
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] != str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    #Then we check the possibilities in D: (1,0) #Upper Left
    new_xlabel = xlabel
    new_ylabel = ylabel
    while True:
        
        new_xlabel += 1
        
        if new_xlabel < 0:
            break
        
        if new_xlabel >7:
            break
       
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] != str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    
    
    
    #Then we check the possibilities in L: (0,-1)  #Lower Left
    new_xlabel = xlabel
    new_ylabel = ylabel
    
    while True:
        new_ylabel -= 1
        
        if new_ylabel <0:
            break
        
        if new_ylabel >7:
            break  
            
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] != str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    #Then we check the possibilities in R: (0,1) #Lower Right
    new_xlabel = xlabel
    new_ylabel = ylabel
    while True:
        
        new_ylabel += 1
        
        if new_ylabel <0:
            break
            
        if new_ylabel >7:
            break   
        
        if chess_position[new_xlabel][new_ylabel] == 0:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
      
        elif str(chess_position[new_xlabel][new_ylabel])[1] != str(chess_position[xlabel][ylabel])[1]:
            tuple_of_possible_positions.append((new_xlabel,new_ylabel))
            break
   
        else:
            break
    
    
    return tuple_of_possible_positions

# Queen

In [14]:
#The Queen moves the same as a rook and a bishop
def possible_queen_positions(initial_square,chess_position):
    
    possible_queen_positions = possible_rook_positions(initial_square,chess_position) + \
    possible_bishop_positions(initial_square,chess_position)
    
    return possible_queen_positions

# White Pawns


In [15]:
#To account for enpassant, we shall 
def enpassant():
    if True:
        return 'y'
    if False:
        return False

In [16]:
#Here the direction Matters
def possible_white_pawn_positions(initial_square,chess_position, enpassant = False, Promotion = 'Queen'):
    tuple_of_possible_positions = []
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
    
    tuple_of_possible_positions = []
    #Pawn can move one or two places if it is yet to move
    if xlabel == 6:
        if chess_position[xlabel -1 ][ylabel] == 0:
            tuple_of_possible_positions.append((xlabel-1,ylabel))
            if chess_position[xlabel -2 ][ylabel] ==0:
                tuple_of_possible_positions.append((xlabel-2,ylabel))
            
    #If pawn has moved it can only move one piece forward
    if xlabel != 6:
        if chess_position[xlabel -1 ][ylabel] == 0:
            tuple_of_possible_positions.append((xlabel-1,ylabel))

                                  

    #Capturing pieces forward left                                                                                  
    if chess_position[xlabel -1][ylabel-1] != 0:
        if str(chess_position[xlabel][ylabel])[1] != str(chess_position[xlabel -1 ][ylabel -1])[1]:
            tuple_of_possible_positions.append((xlabel-1,ylabel-1))
           
        
    #Capturing pieces forward right                                                                                  
    if chess_position[xlabel -1][ylabel+1] != 0:
        if str(chess_position[xlabel][ylabel])[1] != str(chess_position[xlabel -1 ][ylabel +1])[1]:
            tuple_of_possible_positions.append((xlabel-1,ylabel+1))                                      
    
    #EN-Passant 
    #Note to determine if enpassant is permetting i will have a function enpassant, either outputting False or the rank permitted                                          
    if xlabel == 3:
        if enpassant != False:
            tuple_of_possible_positions.append((2,enpassant()))
    return tuple_of_possible_positions 

In [17]:
Board[5][1] = 11
Board[4][5] = 11
Board

array([[21, 31, 41, 51, 61, 41, 31, 21],
       [11, 11, 11, 11, 11, 11, 11, 11],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 11,  0,  0],
       [ 0, 11,  0,  0,  0,  0,  0,  0],
       [10, 10, 10, 10, 10, 10, 10, 10],
       [20, 30, 40, 50, 60, 40, 30, 20]])

In [18]:
possible_white_pawn_positions('60',Board)

[(5, 0), (4, 0), (5, 1)]

# Black pawns

In [27]:
def possible_black_pawn_positions(initial_square,chess_position, enpassant = False, Promotion = 'Queen'):
    tuple_of_possible_positions = []
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
    
    #Start we white pawns
    tuple_of_possible_positions = []
    #Pawn can move one or two places if it is yet to move
    if xlabel == 1:
        if chess_position[xlabel +1 ][ylabel] == 0:
            tuple_of_possible_positions.append((xlabel+1,ylabel))
            if chess_position[xlabel +2 ][ylabel] ==0:
                tuple_of_possible_positions.append((xlabel+2,ylabel))
    ####
    #If pawn has moved it can only move one piece forward
    if xlabel != 1:
        if chess_position[xlabel +1 ][ylabel] == 0:
            tuple_of_possible_positions.append((xlabel+1,ylabel))
    
    #Capturing pieces forward left                                                                                  
    if chess_position[xlabel +1 ][ylabel+1] != 0:
        if str(chess_position[xlabel][ylabel])[1] != str(chess_position[xlabel +1 ][ylabel +1])[1]:
            tuple_of_possible_positions.append((xlabel-1,ylabel-1))
            
        
    #Capturing pieces forward right                                                                                  
    if chess_position[xlabel +1 ][ylabel-1] != 0:
        if str(chess_position[xlabel][ylabel])[1] != str(chess_position[xlabel +1 ][ylabel -1])[1]:
            tuple_of_possible_positions.append((xlabel+1,ylabel-1))                                      
    
                                               
  
        
    return tuple_of_possible_positions    

In [28]:
Board

array([[21, 31, 41, 51, 61, 41, 31, 21],
       [11, 11, 11, 11, 11, 11, 11, 11],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 11,  0,  0],
       [ 0, 11,  0,  0,  0,  0,  0,  0],
       [10, 10, 10, 10, 10, 10, 10, 10],
       [20, 30, 40, 50, 60, 40, 30, 20]])

In [29]:
possible_black_pawn_positions('13',Board)

[(2, 3), (3, 3)]

In [ ]:
#As a precursor

In [ ]:
#As a precursor to generating allowable king moves, we make a list of every 
#Square that the opponents pieces are attacking
def Squares_Black_attacking(position):
    attacking_squares = []
    
    
    
    
    
    
    
    
    
    
    
    
    
def Squares_White_attacking(position):

In [ ]:
def possible_King_positions(initial_square, position, Castle_avaible_kingside = False, Castle_avaible_queenside = False):
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
    #Possible Kings moves
    x_addition = np.array([-1,-1,-1,0,0,1,1,1])
    y_addition = np.array([-1,0,1,-1,1,-1,0,+1])
    possible_x = x_addition + xlabel
    possible_y = y_addition + ylabel
    tuple_of_possible_positions = list(zip(possible_x,possible_y))
    
    #But the King Can't mobve into check, so we remove all values that correspond going to check 
    
    
    
    

In [ ]:
#First we write a function to alter our Board
def move_a_piece(initial_square, target_square, chess_position): #Enter a digit number corresponding to our Board array
    if initial_square not in Allowable_squares:
        print('target_square format incorrect')
    elif target_square not in Allowable_squares:
        print('target_square format incorrect')
    else:
        initial_square_index1, initial_square_index2 = int(initial_square[0]) , int(initial_square[1])
        target_square_index1, target_square_index2 = int(target_square[0]) , int(target_square[1])
        print(chess_position[initial_square_index1],chess_position[initial_square_index1])
    

In [ ]:
'01'[0]

In [ ]:
move_a_piece('01', '02')
    

In [ ]:
game_over = False
turn ==0:
while not game_over:
    if turn == 0:
        move == input()
    
    if turn == 0:
        